In [16]:
ROOT_PATH = "/users/jason wu/downloads/GeNeVA-v1/i-CLEVR/scenes/"

In [17]:
import os
import os.path
import joblib

keys = [os.path.join(ROOT_PATH, f) for f in os.listdir(ROOT_PATH) if f.endswith(".json")]

In [18]:
CACHE_DIR = "/users/jason wu/downloads/GeNeVA-v1/i-CLEVR/cache_scenes"

In [19]:
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [20]:
keys[0]

'/users/jason wu/downloads/GeNeVA-v1/i-CLEVR/scenes/CLEVR_test_000000.json'

In [21]:
import torch
import torch.functional as F

In [22]:
def one_hot(idx, size):
    l = [0 for _ in range(size)]
    l[idx] = 1
    return l

In [23]:
# 8 colors, 3 shape
colormap={'red':0,'green':1,'blue':2,'cyan':3,'brown':4,'gray':5,'purple':6,'yellow':7}
shapemap={'sphere':0, 'cube':1, 'cylinder':2}

In [24]:
def encode_obj(o_pos, o_color, o_shape):
    rep = []
    rep.extend(o_pos)
    rep.extend(one_hot(colormap[o_color], len(colormap)))
    rep.extend(one_hot(shapemap[o_shape], len(shapemap)))
    return rep

In [25]:
from tqdm import tqdm
import json

In [26]:
for key in tqdm(keys):
    filename = key.split("/")[-1].split(".")[0]
    with open(key, 'r') as f:
        key_json = json.load(f)
    objs = key_json['objects']
    last = []
    for i in range(len(objs)):
        subkey = filename + "_" + str(i)
        cache_file = os.path.join(CACHE_DIR, subkey + ".pkl")
        obj = objs[i]
        obj_pos = obj['pixel_coords'][:2]
        obj_color = obj['color']
        obj_shape = obj['shape']
        obj_rep = encode_obj(obj_pos, obj_color, obj_shape)
        start_scene = last[:]
        last.append(obj_rep)
        curr_scene = last[:]
        combined = [start_scene, curr_scene]
        joblib.dump(combined, cache_file)

100%|████████████████████████████████████████████████████████████████████████████| 10003/10003 [01:44<00:00, 95.99it/s]
